# SmartFigure / plotting helper — illustrated tour

This notebook demonstrates the **student-facing plotting helper** exposed by `gu_toolkit.setup()`
(the `SmartFigure` API).

It is organized as a set of short, runnable examples with notes about:
- **what it does**
- **what parameters mean**
- **what errors look like** (and how to fix them)

> **Environment note:** the interactive widget uses **ipywidgets + Plotly FigureWidget**.
> Google Colab is intentionally **not supported** by this module (it raises a helpful error).


In [1]:
import gu_toolkit
gu_toolkit.setup()   # injects SymPy + plugin exports into notebook namespace


🔧 Initializing GU Toolkit...
✓ Matplotlib backend set to 'widget'
[gu_toolkit] Skip (disabled) gu_toolkit.plugins.example
[gu_toolkit] Loaded gu_toolkit.plugins.NamedFunction (exports=1, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartException.SmartException (exports=1, hook=True)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartFigure.SmartFigure (exports=9, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartParameters.SmartParameters (exports=2, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.numpify (exports=1, hook=False)
✅ Smart Exception Handler Activated.
✅ Smart Exception Handler Activated.
[gu_toolkit] Exported 948 names into the notebook namespace.
🎓 GU Toolkit Ready.


## 0. Sanity check: what `setup()` injected

You should now have (at least):
- `sp` (SymPy) or `sympy`
- `SmartFigure`
- `VIEWPORT` (a sentinel meaning “inherit from the figure viewport”)
- (optionally) `Style`, `GuideError`, etc.


In [2]:
# Grab injected SymPy and SmartFigure without importing them explicitly.

assert sp is not None, "Expected `gu_toolkit.setup()` to inject SymPy as `sp` or `sympy`."
assert SmartFigure is not None, "Expected `gu_toolkit.setup()` to inject `SmartFigure`."
assert VIEWPORT is not None, "Expected `gu_toolkit.setup()` to inject `VIEWPORT`."


## 1. Quickstart: first figure and first plot

Key points from the implementation:
- `SmartFigure(show_now=False)` constructs the controller but does **not** display immediately.
- Accessing `fig.widget` builds the Plotly FigureWidget lazily and returns an `ipywidgets.VBox`.
- `fig.plot(expr, ...)` registers a plot and (if the widget exists) triggers compute + draw.


In [3]:

fig = SmartFigure(show_now=False)

fig.widget  # display the widget in Jupyter


    'data': [],
    'layout': {'margin': {'b': 40, 'l': 40, 'r': 20, 't': 20},
 …

In [4]:

p_sin = fig.plot(sin(x), name="sin(x)")

### 1.1 Adding more plots

Each call to `fig.plot(...)` adds (or updates) a *named* plot:
- if `name` is new → create a new Plot
- if `name` already exists → update that existing Plot in-place

Styles can be passed via a `style={...}` dictionary.


In [5]:
p_cos = fig.plot(
    sp.cos(x),
    name="cos(x)",
    style={"color": "red", "width": 3, "linestyle": "--", "opacity": 0.2},
)


## 2. Plot names: auto-naming and updating by name

If you do not provide a `name`, SmartFigure uses `f_1`, `f_2`, ...

If you call `fig.plot(..., name=existing_name)`, the existing plot is updated instead of creating
a second plot with the same name.


In [6]:
p_auto = fig.plot(sp.sin(2 * x))  # no name -> auto name like 'f_1'
print("auto plot name:", p_auto.name)

# Update the existing 'sin(x)' plot in-place:
p_sin_2 = fig.plot(sp.sin(x) / (1 + x**2), name="sin(x)")  # same name as earlier
print("updated object identity preserved:", p_sin_2 is p_sin)


auto plot name: f_1
updated object identity preserved: True


## 3. Viewport controls: `x_range` and `y_range`

- `fig.x_range = (xmin, xmax)` updates the viewport **and recomputes all plots**.
- `fig.y_range = (ymin, ymax)` updates the y-axis range **but does not trigger recomputation**
  (it only changes the displayed axis range).

This matches the current Phase 1 behavior in the code.


In [7]:
# Zoom in: this will resample all plots over the new x-range.
fig.x_range = (-2, 2)


In [8]:
# Change y-range: this updates the display range but does not resample y-values.
fig.y_range = (-1.5, 1.5)


## 4. Domains and sampling

Two separate concepts:

### 4.1 Viewport (`fig.x_range`)
The figure maintains the *view window* where it samples/plots.

### 4.2 Plot domain (`domain=...`)
Each plot can restrict where it is evaluated:
- `domain=VIEWPORT` means “use the full `fig.x_range`”
- `domain=(a, b)` means “evaluate only on `fig.x_range ∩ (a, b)`”
- bounds can be unbounded with `None`: e.g. `domain=(None, 0)`.

### 4.3 Samples
- `SmartFigure(samples=N)` sets a *global* default sample count.
- each plot can override with `samples=...`
- `samples=VIEWPORT` means “inherit the global sample count”.


In [9]:
# A plot with a restricted domain (intersection with viewport).
p_tan = fig.plot(
    sp.tan(x),
    name="tan(x) on (-1, 1)",
    domain=(-1, 1),
    style={"color": "green"},
)

# Unbounded domain example: only the left half-line, intersected with viewport.
p_left = fig.plot(
    sp.exp(x),
    name="exp(x) on (-∞, 0]",
    domain=(None, 0),
    style={"linestyle": ":", "opacity": 0.8},
)



### 4.4 Per-plot samples override

Here we create a second figure with a small global sample count, then override one plot to use
many more points (useful for high-frequency curves).


In [ ]:
fig2 = SmartFigure(show_now=False, samples=150)
x2 = x  # same symbol

p_low = fig2.plot(sp.sin(x2), name="150 samples (global default)")
p_hi = fig2.plot(sp.sin(25 * x2), name="2000 samples (override)", samples=2000)

fig2.widget


## 5. Styling and reactive updates

The `Style` model supports (with aliases):
- `color`
- `width` (aliases: `lw`, `linewidth`)
- `opacity` (alias: `alpha`)
- `linestyle` in `{"-", "--", ":", "-."}` (alias: `ls`)
- `visible`

After a plot is created, you can mutate style *reactively*:

```python
p.style.color = "purple"     # triggers backend update
p.style.width = 5
p.visible = False            # also reactive (convenience)
```


In [ ]:
# Toggle visibility and tweak style live.
p_cos.style.color = "purple"
p_cos.style.width = 5
p_cos.style.opacity = 0.55
p_cos.style.linestyle = "-."

fig.widget


In [ ]:
# Hide/show via the Plot convenience property:
p_tan.visible = False
fig.widget


In [ ]:
p_tan.visible = True
fig.widget


## 6. Plot names: LaTeX ($...$), unicode, and “pseudo-HTML” characters

The Plotly backend sanitizes trace names to avoid silent rendering failures:
- Outside `$...$` math segments, it escapes `&`, `<`, `>` so they display literally.
- Inside `$...$`, it keeps TeX as-is for MathJax.
- If you include an unmatched `$`, it raises a helpful error.

Examples below use a fresh figure to keep names simple.


In [ ]:
fig3 = SmartFigure(show_now=False, x_range=(-6, 6), y_range=(-2, 2))

# LaTeX in the legend:
p1 = fig3.plot(sp.sin(x), name="$\sin(x)$")

# Literal angle brackets and ampersands (escaped outside math):
p2 = fig3.plot(sp.cos(x), name="Angle <x> & <y>")

# A literal dollar sign outside math: write it as '\$'
p3 = fig3.plot(sp.sin(x) + 0.5, name="Cost is \$5")

fig3.widget


In [ ]:
# Unmatched '$' should raise a GuideError (caught here so the notebook continues).
try:
    fig3.plot(sp.sin(x), name="$sin(x)")  # unmatched '$'
except Exception as e:
    print(type(e).__name__ + ":", e)


## 7. Error handling and common pitfalls

This section intentionally triggers errors to show how the helper protects students.

### 7.1 Multiple free symbols
If an expression has multiple free symbols and you don’t specify `symbol=...`, it raises a `GuideError`.


In [ ]:
y = sp.Symbol("y")

try:
    fig3.plot(x + y, name="x + y (should error)")
except Exception as e:
    print(type(e).__name__ + ":", e)


### 7.2 Non-numeric expressions (extra parameters)

Even if you specify `symbol=x`, an expression like `sin(a*x)` still contains the free symbol `a`.
The compiled numerical function only receives `x`, so evaluation fails unless you substitute `a`
with a number first.


In [ ]:
a = sp.Symbol("a")

try:
    fig3.plot(sp.sin(a * x), name="sin(a*x) (should error)", symbol=x)
except Exception as e:
    print(type(e).__name__ + ":", e)

# Fix: substitute parameters with numbers.
p_param = fig3.plot(sp.sin(a * x).subs(a, 2), name="sin(2x) (after substitution)", symbol=x)
fig3.widget


### 7.3 Complex-valued expressions

The implementation rejects complex-valued outputs with a non-negligible imaginary part.


In [ ]:
try:
    fig3.plot(sp.exp(sp.I * x), name="exp(i x) (complex; should error)")
except Exception as e:
    print(type(e).__name__ + ":", e)


### 7.4 Invalid `domain=...`

`domain` must be:
- `VIEWPORT`, or
- a 2-tuple `(min, max)` where each entry is float-like (or `None`).


In [ ]:
try:
    fig3.plot(sp.sin(x), name="bad domain", domain=("left", "right"))
except Exception as e:
    print(type(e).__name__ + ":", e)


### 7.5 Invalid `style=...`

Unknown style keys (or invalid values) raise a helpful error.

Allowed linestyle values are exactly one of: `"-"`, `"--"`, `":"`, `"-."`.


In [ ]:
try:
    fig3.plot(sp.sin(x), name="bad style key", style={"not_a_style_key": 123})
except Exception as e:
    print(type(e).__name__ + ":", e)

try:
    fig3.plot(sp.sin(x), name="bad linestyle", style={"linestyle": "??? "})
except Exception as e:
    print(type(e).__name__ + ":", e)


### 7.6 Unsupported keyword arguments to `fig.plot`

The `plot(...)` method has an explicit signature and does **not** accept arbitrary `**kwargs`.
If you want to pass appearance options, use `style={...}`.


In [ ]:
try:
    # 'color' is not a valid keyword argument; it belongs in style={...}
    fig3.plot(sp.sin(x), name="oops", color="red")
except TypeError as e:
    print("TypeError:", e)


## 8. Managing plots: listing, removing, clearing

- `fig.get_plot_names()` returns a list of names in insertion order.
- `fig.remove(name)` removes a plot by name.
- `fig.clear()` removes everything.


In [ ]:
print("fig3 plot names:", fig3.get_plot_names())

# Remove one plot
fig3.remove("Angle <x> & <y>")
print("after remove:", fig3.get_plot_names())
fig3.widget


In [ ]:
# Clear everything
fig3.clear()
print("after clear:", fig3.get_plot_names())
fig3.widget


## Appendix: Accessing the underlying Plotly FigureWidget (advanced)

`SmartFigure.backend` returns the backend **after** the widget has been constructed.
For Plotly, the backend has a `.fig` attribute (a `plotly.graph_objects.FigureWidget`).

This can be useful for small, advanced tweaks that are not part of the student API.


In [ ]:
# Ensure widget exists, then access the underlying FigureWidget.
_ = fig.widget
backend = fig.backend
print("backend type:", type(backend).__name__)

# If this is PlotlyBackend, it exposes `backend.fig`.
if hasattr(backend, "fig"):
    backend.fig.update_layout(title="SmartFigure (advanced tweak: title added via backend)")
fig.widget
